### 📌Q1. 가장 먼저 학습 데이터를 준비해보도록 하겠습니다. MNIST 데이터셋을 직접 Load해 봅시다. 데이터셋을 로드하고 DataLoader를 구현해보세요.

* DataLoader를 이용해 MNIST 데이터셋을 로드해봅시다.

In [66]:
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [67]:
training_epochs = 15

In [68]:
batch_size = 100

In [69]:
root = './data'

In [79]:
mnist_train = dset.MNIST (root=root, train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dset.MNIST (root=root, train=False, transform=transforms.ToTensor(), download=True)

In [80]:
#data loader 구현
train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False, drop_last=True)

### 📌Q2.데이터가 준비가 되었다면, 이제 그 데이터를 학습할 모델을 구현할 차례입니다. 그후 모델 안의 가중치를 초기화시켜보세요. 입력 데이터 형태에 맞도록 linear한 모델을 구성해보세요.

* MNIST 입력의 크기는 28x28입니다.

* 여기서 구현하는 linear 모델은 입력이 1차원이기 때문에 입력 차원을 맞춰보세요.

In [81]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
linear = torch.nn.Linear(784, 10, bias=True).to(device)

In [82]:
#initialization
torch.nn.init.normal(linear.weight)

C:\Users\User\AppData\Local\Temp\ipykernel_11776\2238913059.py:2: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  torch.nn.init.normal(linear.weight)


Parameter containing:
tensor([[-1.1963,  0.6151, -0.4301,  ...,  0.1879, -0.8384, -0.2137],
        [-0.0841,  1.1740, -1.1685,  ..., -0.0383,  0.1817,  0.1730],
        [ 0.5375, -0.8526, -0.8004,  ..., -0.8893, -0.2328, -0.5297],
        ...,
        [ 1.4213, -0.6979,  0.2195,  ...,  1.0020, -0.4909,  0.2429],
        [ 0.5657, -0.1137, -0.1228,  ..., -0.2837, -0.3430,  0.0146],
        [-0.0469, -1.4833, -0.9512,  ..., -0.4798,  1.5152,  0.4757]],
       requires_grad=True)

### 📌Q3. 위에서 구현한 모델을 학습시키기 위해서는 loss 함수와 opitmizer가 필요합니다. 아래 제시된 loss 함수와 optimizer를 구현해보세요. Loss 함수와 optimizer는 모델 안의 가중치를 업데이트 할 때 사용됩니다.

* 옵티마이저는 SGD, Loss는 Cross Entropy Loss를 사용합니다.

In [83]:
# Loss fn - Cross Entropy Loss
criterion = torch.nn.CrossEntropyLoss().to(device) 

In [84]:
# optimizer - SGD
optimizer = torch.optim.SGD(linear.parameters(), lr = 0.1)

### 📌Q4. 3번 문제까지 해결하셨다면, 이제 학습을 위한 준비는 거의 끝났다고 볼 수 있습니다. 위 구현 함수들을 이용해 학습 Loop를 구현해보세요.

* 위에서 구현한 모델, optimzer, loss fn 등을 이용해 학습을 구현해주세요.

In [88]:
for epoch in range(training_epochs):
    
    for i, (imgs, labels) in enumerate(train_loader):
        imgs, labels = imgs.to(device), labels.to(device)
        imgs = imgs.view(-1, 28 * 28)
        
        outputs = linear(imgs)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        _,argmax = torch.max(outputs, 1)
        accuracy = (labels == argmax).float().mean()
        
    if(i+1)%100 == 0:
        print('Epoch [{}/{}], Step[{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch+1, training_epochs, i+1, len(train_loader), loss.item(), accuracy.item()*100))

Epoch [1/15], Step[600/600], Loss: 0.9098, Accuracy: 77.00%
Epoch [2/15], Step[600/600], Loss: 0.8568, Accuracy: 82.00%
Epoch [3/15], Step[600/600], Loss: 0.9429, Accuracy: 84.00%
Epoch [4/15], Step[600/600], Loss: 0.4608, Accuracy: 91.00%
Epoch [5/15], Step[600/600], Loss: 0.7316, Accuracy: 84.00%
Epoch [6/15], Step[600/600], Loss: 0.5179, Accuracy: 89.00%
Epoch [7/15], Step[600/600], Loss: 0.4712, Accuracy: 87.00%
Epoch [8/15], Step[600/600], Loss: 0.3044, Accuracy: 92.00%
Epoch [9/15], Step[600/600], Loss: 0.4762, Accuracy: 90.00%
Epoch [10/15], Step[600/600], Loss: 0.5087, Accuracy: 90.00%
Epoch [11/15], Step[600/600], Loss: 0.5068, Accuracy: 91.00%
Epoch [12/15], Step[600/600], Loss: 0.3205, Accuracy: 90.00%
Epoch [13/15], Step[600/600], Loss: 0.7154, Accuracy: 83.00%
Epoch [14/15], Step[600/600], Loss: 0.5065, Accuracy: 90.00%
Epoch [15/15], Step[600/600], Loss: 0.2553, Accuracy: 94.00%


### 📌Q5. 학습이 완료되면, 모델이 잘 동작하는지 테스트가 필요합니다. 데이터로드 파트에서 준비했던 테스트 데이터를 이용해 테스트를 진행해봅시다. 아래 테스트 코드를 완성해보세요.

In [91]:
linear.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for i, (imgs, labels) in enumerate(test_loader):
        imgs, labels = imgs.to(device), labels.to(device)
        imgs = imgs.view(-1, 28*28)
        
        outputs = linear(imgs)
        
        _, argmax = torch.max(outputs, 1)
        total += imgs.size(0)
        correct += (labels == argmax).sum().item()
        
        print('Test accuracy for {} images: {:.2f}%'.format(total, correct / total*100))

Test accuracy for 100 images: 92.00%
Test accuracy for 200 images: 89.50%
Test accuracy for 300 images: 90.33%
Test accuracy for 400 images: 88.50%
Test accuracy for 500 images: 87.80%
Test accuracy for 600 images: 86.67%
Test accuracy for 700 images: 87.14%
Test accuracy for 800 images: 87.25%
Test accuracy for 900 images: 87.56%
Test accuracy for 1000 images: 87.00%
Test accuracy for 1100 images: 87.00%
Test accuracy for 1200 images: 86.58%
Test accuracy for 1300 images: 86.08%
Test accuracy for 1400 images: 86.14%
Test accuracy for 1500 images: 85.80%
Test accuracy for 1600 images: 86.00%
Test accuracy for 1700 images: 86.06%
Test accuracy for 1800 images: 86.00%
Test accuracy for 1900 images: 86.16%
Test accuracy for 2000 images: 85.90%
Test accuracy for 2100 images: 85.71%
Test accuracy for 2200 images: 85.77%
Test accuracy for 2300 images: 86.00%
Test accuracy for 2400 images: 86.12%
Test accuracy for 2500 images: 86.16%
Test accuracy for 2600 images: 86.12%
Test accuracy for 270